# CmdStanPy Example Notebook

This notebook demonstrates how to install the [CmdStanPy](https://cmdstanpy.readthedocs.io/en/latest/index.html) toolchain on a Google Colab instance and verify the installation by running the Stan NUTS-HMC sampler on the example model and data which are included with CmdStan. Each code block in this notebook updates the Python environment, therefore you must step through this notebook cell by cell.

In [ ]:
# Load packages used in this notebook
import os
import json
import shutil
import urllib.request
import pandas as pd

Step 1: install CmdStanPy

In [ ]:
# Install package CmdStanPy
!pip install --upgrade cmdstanpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Step 2: download and untar the CmdStan binary for Google Colab instances.

In [ ]:
# Install pre-built CmdStan binary
# (faster than compiling from source via install_cmdstan() function)
tgz_file = 'colab-cmdstan-2.23.0.tar.gz'
tgz_url = 'https://github.com/stan-dev/cmdstan/releases/download/v2.23.0/colab-cmdstan-2.23.0.tar.gz'
if not os.path.exists(tgz_file):
    urllib.request.urlretrieve(tgz_url, tgz_file)
    shutil.unpack_archive(tgz_file)

Step 3: Register the CmdStan install location.

In [ ]:
# Specify CmdStan location via environment variable
os.environ['CMDSTAN'] = './cmdstan-2.23.0'
# Check CmdStan path
from cmdstanpy import CmdStanModel, cmdstan_path
cmdstan_path()

'cmdstan-2.23.0'

The CmdStan installation includes a simple example program `bernoulli.stan` and test data `bernoulli.data.json`. These are in the CmdStan installation directory `examples/bernoulli`.

The program `bernoulli.stan` takes a vector `y` of length `N` containing binary outcomes and uses a bernoulli distribution to estimate `theta`, the chance of success.

In [ ]:
bernoulli_stan = os.path.join(cmdstan_path(), 'examples', 'bernoulli', 'bernoulli.stan')
with open(bernoulli_stan, 'r') as fd:
        print('\n'.join(fd.read().splitlines()))

data { 
  int<lower=0> N; 
  int<lower=0,upper=1> y[N];
} 
parameters {
  real<lower=0,upper=1> theta;
} 
model {
  theta ~ beta(1,1);  // uniform prior on interval 0,1
  y ~ bernoulli(theta);
}


The data file `bernoulli.data.json` contains 10 observations, split between 2 successes (1) and 8 failures (0).

In [ ]:
bernoulli_data = os.path.join(cmdstan_path(), 'examples', 'bernoulli', 'bernoulli.data.json')
with open(bernoulli_data, 'r') as fd:
        print('\n'.join(fd.read().splitlines()))

{
    "N" : 10,
    "y" : [0,1,0,0,0,0,0,0,0,1]
}


The following code test that the CmdStanPy toolchain is properly installed by compiling the example model, fitting it to the data, and obtaining a summary of estimates of the posterior distribution of all parameters and quantities of interest.



In [ ]:
# Run CmdStanPy Hello, World! example
from cmdstanpy import cmdstan_path, CmdStanModel

# Compile example model bernoulli.stan
bernoulli_model = CmdStanModel(stan_file=bernoulli_stan)

# Condition on example data bernoulli.data.json
bern_fit = bernoulli_model.sample(data=bernoulli_data, seed=123)

# Let's try with ADVI algorithm
bern_data = os.path.join(cmdstan_path(), 'examples', 'bernoulli', 'bernoulli.data.json')
bern_vb = bernoulli_model.variational(data=bern_data)
print(bern_vb.column_names)
print(bern_vb.variational_params_dict)
bern_vb.variational_sample.shape

# Print a summary of the posterior sample
# bern_fit.summary()

DEBUG:cmdstanpy:found newer exe file, not recompiling
DEBUG:cmdstanpy:cmd: /content/cmdstan-2.23.0/examples/bernoulli/bernoulli info
cwd: None
DEBUG:cmdstanpy:Command ['/content/cmdstan-2.23.0/examples/bernoulli/bernoulli', 'info']
	error during processing Machine is not on the network
01:07:06 - cmdstanpy - INFO - CmdStan start processing
INFO:cmdstanpy:CmdStan start processing


chain 1 |          | 00:00 Status

chain 2 |          | 00:00 Status

chain 3 |          | 00:00 Status

chain 4 |          | 00:00 Status

DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: 1
DEBUG:cmdstanpy:CmdStan args: ['/content/cmdstan-2.23.0/examples/bernoulli/bernoulli', 'id=1', 'random', 'seed=123', 'data', 'file=cmdstan-2.23.0/examples/bernoulli/bernoulli.data.json', 'output', 'file=/tmp/tmpyd8y2jwn/bernoullipby_q2tk/bernoulli-20230423010706_1.csv', 'method=sample', 'algorithm=hmc', 'adapt', 'engaged=1']
DEBUG:cmdstanpy:idx 1
DEBUG:cmdstanpy:running CmdStan, num_threads: 1
DEBUG:cmdstanpy:CmdStan args: ['/content/cmdstan-2.23.0/examples/bernoulli/bernoulli', 'id=2', 'random', 'seed=123', 'data', 'file=cmdstan-2.23.0/examples/bernoulli/bernoulli.data.json', 'output', 'file=/tmp/tmpyd8y2jwn/bernoullipby_q2tk/bernoulli-20230423010706_2.csv', 'method=sample', 'algorithm=hmc', 'adapt', 'engaged=1']
DEBUG:cmdstanpy:idx 2
DEBUG:cmdstanpy:running CmdStan, num_threads: 1
DEBUG:cmdstanpy:CmdStan args: ['/content/cmdstan-2.23.0/examples/bernoulli/bernoulli', 'id=3', 'random', 'seed=123', 'data', 'file=cmdst

01:07:06 - cmdstanpy - INFO - CmdStan done processing.
INFO:cmdstanpy:CmdStan done processing.
DEBUG:cmdstanpy:runset
RunSet: chains=4, chain_ids=[1, 2, 3, 4], num_processes=4
 cmd (chain 1):
	['/content/cmdstan-2.23.0/examples/bernoulli/bernoulli', 'id=1', 'random', 'seed=123', 'data', 'file=cmdstan-2.23.0/examples/bernoulli/bernoulli.data.json', 'output', 'file=/tmp/tmpyd8y2jwn/bernoullipby_q2tk/bernoulli-20230423010706_1.csv', 'method=sample', 'algorithm=hmc', 'adapt', 'engaged=1']
 retcodes=[0, 0, 0, 0]
 per-chain output files (showing chain 1 only):
 csv_file:
	/tmp/tmpyd8y2jwn/bernoullipby_q2tk/bernoulli-20230423010706_1.csv
 console_msgs (if any):
	/tmp/tmpyd8y2jwn/bernoullipby_q2tk/bernoulli-20230423010706_0-stdout.txt
DEBUG:cmdstanpy:Chain 1 console:
method = sample (Default)
  sample
    num_samples = 1000 (Default)
    num_warmup = 1000 (Default)
    save_warmup = 0 (Default)
    thin = 1 (Default)
    adapt
      engaged = 1 (Default)
      gamma = 0.050000000000000003 (Def


('lp__', 'log_p__', 'log_g__', 'theta')
OrderedDict([('lp__', 0.0), ('log_p__', 0.0), ('log_g__', 0.0), ('theta', 0.260083)])


(1000, 4)

In [ ]:
# Print a summary of the posterior sample
bern_fit.summary()

DEBUG:cmdstanpy:cmd: cmdstan-2.23.0/bin/stansummary --percentiles= 5,50,95 --sig_figs=6 --csv_file=/tmp/tmpyd8y2jwn/stansummary-bernoulli-07y1208q.csv /tmp/tmpyd8y2jwn/bernoullipby_q2tk/bernoulli-20230423010706_1.csv /tmp/tmpyd8y2jwn/bernoullipby_q2tk/bernoulli-20230423010706_2.csv /tmp/tmpyd8y2jwn/bernoullipby_q2tk/bernoulli-20230423010706_3.csv /tmp/tmpyd8y2jwn/bernoullipby_q2tk/bernoulli-20230423010706_4.csv
cwd: None


,Mean,MCSE,StdDev,5%,50%,95%,N_Eff,N_Eff/s,R_hat
lp__,-7.269880,0.018486,0.737313,-8.845200,-6.979330,-6.749980,1590.87,15715.4,1.00195
theta,0.243855,0.003029,0.116829,0.076252,0.230079,0.452028,1487.34,14692.7,1.00197
